# Bike Share System

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# import functions from modsim
from modsim import *

In [7]:
def step(state, p1, p2):
    """Simulate one time step.
    
    state: bikeshare State object
    p1: probability of an Olin->Wellesley ride
    p2: probability of a Wellesley->Olin ride
    """
    if flip(p1):
        bike_to_wellesley(state)
    
    if flip(p2):
        bike_to_olin(state)
        
def bike_to_olin(state):
    """Move one bike from Wellesley to Olin.
    
    state: bikeshare State object
    """
    if state.wellesley == 0:
        state.wellesley_empty += 1
        return
    state.wellesley -= 1
    state.olin += 1
    
def bike_to_wellesley(state):
    """Move one bike from Olin to Wellesley.
    
    state: bikeshare State object
    """
    
    if state.olin == 0:
        state.olin_empty += 1
        return
    state.olin -= 1
    state.wellesley += 1

In [8]:
def run_simulation(state, p1, p2, num_steps, do_plot=True):
    """Simulate the given number of time steps.
    
    state: State object
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    num_steps: number of time steps
    """
    results_olin = TimeSeries()
    results_olin[0] = state.olin
    results_wellesley = TimeSeries()
    results_wellesley[0] = state.wellesley
    results_olin_unsatisfied = TimeSeries()
    results_olin_unsatisfied[0] = state.olin_empty
    results_wellesley_unsatisfied = TimeSeries()
    results_wellesley_unsatisfied[0] = state.wellesley_empty
    
    for i in range(num_steps):
        step(state, p1, p2)
        results_olin[i+1] = state.olin
        results_wellesley[i+1] = state.wellesley
        results_olin_unsatisfied[i+1] = state.olin_empty
        results_wellesley_unsatisfied[i+1] = state.wellesley_empty
        
    if (do_plot):
        fig, ax = plt.subplots(1,2,figsize=(12,6))
        ax[0].plot(results_olin, label='Olin')
        ax[0].plot(results_wellesley, label='Wellesley')
        ax[0].set_xlabel('Time step (min)')
        ax[0].set_ylabel('Number of bikes')
        ax[0].legend()
        ax[1].plot(results_olin_unsatisfied, label='Olin')
        ax[1].plot(results_wellesley_unsatisfied, label='Wellesley')
        ax[1].set_xlabel('Time step (min)')
        ax[1].set_ylabel('Number of unsatisfied customers')
        ax[1].legend()
        plt.show()
    
    return results_olin_unsatisfied[num_steps-1] + results_wellesley_unsatisfied[num_steps-1]

In [9]:
p_olin_to_wellesley = 0.3
p_wellesley_to_olin = 0.2

N_slots = 100

nstart = []
tu_avg = []

num_sims = 100

for N_start_olin in range(N_slots+1):
    nstart.append(N_start_olin)
    N_start_wellesley = N_slots - N_start_olin

    print(N_start_olin, N_start_wellesley)

    tu_sum = 0
    
    for j in range(num_sims):
        bikeshare = State(olin=N_start_olin, wellesley=N_start_wellesley,
                  olin_empty=0, wellesley_empty=0)
        do_plot = False
        total_unsatisfied = run_simulation(bikeshare, p_wellesley_to_olin, p_olin_to_wellesley, 100, do_plot)

        #print("Total unsatisfied customers: ", j, total_unsatisfied)
        tu_sum += total_unsatisfied
    
    print("Average unsatisfied customers: ", tu_sum/(1.0*num_sims))
    tu_avg.append(tu_sum/(1.0*num_sims))
    


0 100
Average unsatisfied customers:  1.4
1 99
Average unsatisfied customers:  0.89
2 98
Average unsatisfied customers:  0.55
3 97
Average unsatisfied customers:  0.5
4 96
Average unsatisfied customers:  0.11
5 95
Average unsatisfied customers:  0.36
6 94


KeyboardInterrupt: 

In [ ]:
tu_avg_error = 0.1*np.array(tu_avg)

fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.errorbar(nstart, tu_avg, tu_avg_error, label='Average unsatisfied customers')
ax.set_xlabel('Number of starting bikes at Olin')
ax.set_ylabel('Average number of unsatisfied customers')
ax.legend()

## Modeling a Bike Share System

Imagine a bike share system for students traveling between Olin College and Wellesley College, which are about three miles apart in eastern Massachusetts.

Suppose the system contains 12 bikes and two bike racks, one at Olin and one at Wellesley, each with the capacity to hold 12 bikes.

As students arrive, check out a bike, and ride to the other campus, the number of bikes in each location changes. In the simulation, we'll need to keep track of where the bikes are. To do that, we'll use a function called `State`, which is defined in the ModSim library.

## Under the Hood

This section contains additional information about the functions we've used and pointers to their documentation.

You don't need to know anything in this section, so if you are already feeling overwhelmed, you might want to skip it.
But if you are curious, read on.

`State` and `TimeSeries` objects are based on the `Series` object defined by the Pandas library.
The documentation is at <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html>.

`Series` objects provide their own `plot` function, which is why we call it like this:

```
results.plot()
```

Instead of like this:

```
plot(results)
```

You can read the documentation of `Series.plot` at <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.html>.

`decorate` is based on Matplotlib, which is a widely used plotting library for Python.  Matplotlib provides separate functions for `title`, `xlabel`, and `ylabel`.
`decorate` makes them a little easier to use.
For the list of keyword arguments you can pass to `decorate`, see <https://matplotlib.org/3.2.2/api/axes_api.html?highlight=axes#module-matplotlib.axes>.

The `flip` function uses NumPy's `random` function to generate a random number between 0 and 1, then returns `True` or `False` with the given probability.

You can get the source code for `flip` (or any other function) by running the following cell.

In [ ]:
source_code(flip)

## 3 different stops
3 stops: A, B, and C -> 6 possible routes: AB, AC, BA, BC, CA, CB

In [2]:
def step(state, pAB, pAC, pBA, pBC, pCA, pCB):
    """Simulate one time step.
    
    state: bikeshare State object
    pAB: probability of an A->B ride
    pAC: probability of an A->C ride
    pBA: probability of a B->A ride
    pBC: probability of a B->C ride
    pCA: probability of a C->A ride
    pCB: probability of a C->B ride
    """
    if flip(pAB):
        bike_A_to_B(state)
    
    if flip(pAC):
        bike_A_to_C(state)
    
    if flip(pBA):
        bike_B_to_A(state)
    
    if flip(pBC):
        bike_B_to_C(state)
    
    if flip(pCA):
        bike_C_to_A(state)
        
    if flip(pCB):
        bike_C_to_B(state)

In [3]:
def bike_A_to_B(state):
    """Move one bike from A to B.
    
    state: bikeshare State object
    """
    if state.A == 0:
        state.A_empty += 1
        return
    state.A -= 1
    state.B += 1

def bike_A_to_C(state):
    """Move one bike from A to C.
    
    state: bikeshare State object
    """
    if state.A == 0:
        state.A_empty += 1
        return
    state.A -= 1
    state.C += 1
    
def bike_B_to_A(state):
    """Move one bike from B to A.
    
    state: bikeshare State object
    """
    if state.B == 0:
        state.B_empty += 1
        return
    state.B -= 1
    state.A += 1
    
def bike_B_to_C(state):
    """Move one bike from B to C.
    
    state: bikeshare State object
    """
    if state.B == 0:
        state.B_empty += 1
        return
    state.B -= 1
    state.C += 1

def bike_C_to_A(state):
    """Move one bike from C to A.
    
    state: bikeshare State object
    """
    if state.C == 0:
        state.C_empty += 1
        return
    state.C -= 1
    state.A += 1

def bike_C_to_B(state):
    """Move one bike from C to B.
    
    state: bikeshare State object
    """
    if state.C == 0:
        state.C_empty += 1
        return
    state.C -= 1
    state.B += 1

In [4]:
def run_simulation(state, pAB, pAC, pBA, pBC, pCA, pCB, num_steps, do_plot=True):
    """Simulate the given number of time steps.
    
    state: State object
    pAB: probability of an A->B ride
    pAC: probability of an A->C ride
    pBA: probability of a B->A ride
    pBC: probability of a B->C ride
    pCA: probability of a C->A ride
    pCB: probability of a C->B ride
    num_steps: number of time steps
    """
    results_A = TimeSeries()
    results_A[0] = state.A
    results_B = TimeSeries()
    results_B[0] = state.B
    results_C = TimeSeries()
    results_C[0] = state.C
    results_A_unsatisfied = TimeSeries()
    results_A_unsatisfied[0] = state.A_empty
    results_B_unsatisfied = TimeSeries()
    results_B_unsatisfied[0] = state.B_empty
    results_C_unsatisfied = TimeSeries()
    results_C_unsatisfied[0] = state.C_empty
    
    for i in range(num_steps):
        step(state, pAB, pAC, pBA, pBC, pCA, pCB)
        results_A[i+1] = state.A
        results_B[i+1] = state.B
        results_C[i+1] = state.C
        results_A_unsatisfied[i+1] = state.A_empty
        results_B_unsatisfied[i+1] = state.B_empty
        results_C_unsatisfied[i+1] = state.C_empty
        
    if (do_plot):
        fig, ax = plt.subplots(1,2,figsize=(12,6))
        ax[0].plot(results_A, label='A')
        ax[0].plot(results_B, label='B')
        ax[0].plot(results_C, label='C')
        ax[0].set_xlabel('Time step (min)')
        ax[0].set_ylabel('Number of bikes')
        ax[0].legend()
        ax[1].plot(results_A_unsatisfied, label='A')
        ax[1].plot(results_B_unsatisfied, label='B')
        ax[1].plot(results_C_unsatisfied, label='C')
        ax[1].set_xlabel('Time step (min)')
        ax[1].set_ylabel('Number of unsatisfied customers')
        ax[1].legend()
        plt.show()
    
    return results_A_unsatisfied[num_steps-1] + results_B_unsatisfied[num_steps-1] + results_C_unsatisfied[num_steps-1]

In [5]:
pAB = 0.3
pAC = 0.4
pBA = 0.2
pBC = 0.35
pCA = 0.1
pCB = 0.15

N_slots = 100

nstart = []
tu_avg = []

num_sims = 100

for N_start_A in range(0,101,10):
    # print(N_start_A)
    for N_start_B in range(0,101-N_start_A,10):
        N_start_C = 100 - N_start_A - N_start_B
        print(f"A: {N_start_A}, B: {N_start_B}, C: {N_start_C}")
        
        tu_sum = 0
    
        for j in range(num_sims):
            bikeshare = State(A=N_start_A, B=N_start_B,C=N_start_C,
                      A_empty=0, B_empty=0, C_empty=0)
            do_plot = False
            total_unsatisfied = run_simulation(bikeshare, pAB, pAC, pBA, pBC, pCA, pCB, 100, do_plot)
    
            #print("Total unsatisfied customers: ", j, total_unsatisfied)
            tu_sum += total_unsatisfied
    
        print("Average unsatisfied customers: ", tu_sum/(1.0*num_sims))
        tu_avg.append(tu_sum/(1.0*num_sims))

A: 0, B: 0, C: 100
Average unsatisfied customers:  81.5
A: 0, B: 10, C: 90
Average unsatisfied customers:  64.84
A: 0, B: 20, C: 80
Average unsatisfied customers:  51.28
A: 0, B: 30, C: 70
Average unsatisfied customers:  41.83
A: 0, B: 40, C: 60
Average unsatisfied customers:  39.47
A: 0, B: 50, C: 50
Average unsatisfied customers:  40.29
A: 0, B: 60, C: 40
Average unsatisfied customers:  40.59
A: 0, B: 70, C: 30
Average unsatisfied customers:  40.12
A: 0, B: 80, C: 20
Average unsatisfied customers:  41.2
A: 0, B: 90, C: 10
Average unsatisfied customers:  39.24
A: 0, B: 100, C: 0
Average unsatisfied customers:  40.96
A: 10, B: 0, C: 90
Average unsatisfied customers:  65.72
A: 10, B: 10, C: 80
Average unsatisfied customers:  50.04
A: 10, B: 20, C: 70
Average unsatisfied customers:  38.27
A: 10, B: 30, C: 60
Average unsatisfied customers:  30.84
A: 10, B: 40, C: 50
Average unsatisfied customers:  30.08
A: 10, B: 50, C: 40
Average unsatisfied customers:  31.07
A: 10, B: 60, C: 30
Average 

Lowest average unsatisfied customers is 0.02 for starting values of A: 60, B: 40, C: 0. The best distribution of bikes is around this spot in A,B,C space.